In [3]:
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import pandas as pd

In [8]:
def get_chemblID(pubchemID):
    comp_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound'
    comp_res = requests.get(f'{comp_url}/{pubchemID}/JSON/?response_type=display')
    soup = comp_res.json()
    try:
        a = soup['Record']
        for b in a['Section']:
            if b['TOCHeading'] == 'Names and Identifiers':
                for c in b['Section']:
                    if c['TOCHeading'] == 'Synonyms':
                        for d in c['Section']:
                            if d['TOCHeading'] == 'Depositor-Supplied Synonyms':
                                for i in d['Information'][0]['Value']['StringWithMarkup']:
                                    if i['String'].startswith('CHEMBL'):
                                        return i['String']
                                break
                        break
                break
    except:
        return None

In [27]:
def get_maxpahse(chembl_id):
    try:
        chem_api = f'https://www.ebi.ac.uk/chembl/api/data/drug/{chembl_id}?format=json'
        res = requests.get(chem_api)
        res = res.json()
        return res.get('development_phase')
    except:
        return None

In [22]:
raw_clinical = pd.read_csv('./PubChem_compound_text_covid-19 clinicaltrials.csv')
raw_clinical = raw_clinical[['cid','cmpdname']].drop_duplicates('cid')

In [23]:
for i in raw_clinical.index:
    raw_clinical.at[i,'ChEMBL ID'] = get_chemblID(raw_clinical.loc[i,'cid'])

In [30]:
raw_clinical['Clinical_phase'] = 0
for i in raw_clinical.index:
    raw_clinical.at[i,'Clinical_phase'] = get_maxpahse(raw_clinical.loc[i,'ChEMBL ID'])

In [32]:
raw_clinical.to_csv('./clinicaltrial.csv',index=False)

---

In [67]:
raw_clinical = pd.read_csv('./PubChem_compound_text_covid-19 clinicaltrials.csv')
raw_clinical = raw_clinical[['cid','cmpdname']].drop_duplicates('cid')

In [68]:
cid2chembl = pd.read_csv('./clinicaltri_cid2chembl.txt',sep='\t',header=None)
synonyms2chembl = pd.read_csv('./clinicaltri_synonyms2chembl.txt',sep='\t',header=None)


In [69]:
cid2chembl = cid2chembl.drop_duplicates(0)
cid2chembl = pd.Series(index=cid2chembl[0].values, data=cid2chembl[1].values)
synonyms2chembl = synonyms2chembl.drop_duplicates(0)
synonyms2chembl = pd.Series(index=synonyms2chembl[0].values, data=synonyms2chembl[1].values)

In [70]:
raw_clinical['chembl1'] = raw_clinical['cid'].map(cid2chembl)
raw_clinical['chembl2'] = raw_clinical['cmpdname'].map(synonyms2chembl)


In [71]:
raw_clinical['ChEMBL'] = raw_clinical['chembl1']
raw_clinical['ChEMBL'] = raw_clinical['chembl2'].where(((raw_clinical['chembl1'].isnull())&(raw_clinical['chembl2'].notnull())),raw_clinical['ChEMBL'])
raw_clinical = raw_clinical[raw_clinical['ChEMBL'].notnull()][['cid','cmpdname','ChEMBL']]
raw_clinical.to_csv('./clinicaltrial.csv', index=False)

In [73]:
raw_clinical['Clinical_phase'] = 0
for i in raw_clinical.index:
    raw_clinical.at[i,'Clinical_phase'] = get_maxpahse(raw_clinical.loc[i,'ChEMBL'])

In [75]:
raw_clinical['Clinical_phase'] = raw_clinical['Clinical_phase'].fillna(0)

In [76]:
raw_clinical.to_csv('./clinicaltrial.csv', index=False)